In [50]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os
from google.cloud import bigquery

html = requests.get('https://runrepeat.com/nba-revenue-statistics')
soup = BeautifulSoup(html.content, 'html.parser')

tables = soup.find_all('table', class_='table table-bordered')

In [51]:
dataframes = []
filenames = ['rev_by_team', 'ticket_revenue', 'operating_income', 'jersey_patch_sponsorships']

for table in tables:
    df = pd.read_html(str(table))[0]  # Use [0] because read_html returns a list of DataFrames
    header = df.iloc[0]
    header[0] = 'Team'
    df = df[1:]
    df.columns = header
    dataframes.append(df)
    
dataframes[-1].rename(columns={'Jersey patch annual revenue ($ million)': 'jersey_rev_mil'}, inplace=True)

/var/folders/x3/42y9jxy964lg3yyk_9zhss280000gn/T/ipykernel_68513/910569192.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]  # Use [0] because read_html returns a list of DataFrames
/var/folders/x3/42y9jxy964lg3yyk_9zhss280000gn/T/ipykernel_68513/910569192.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]  # Use [0] because read_html returns a list of DataFrames
/var/folders/x3/42y9jxy964lg3yyk_9zhss280000gn/T/ipykernel_68513/910569192.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]  # Use [0] because read_html 

In [52]:
# set credentials to get access to BQ proj
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/Users/bennett/PycharmProjects/ba775/key.json'

# create a client obj to do stuff
client = bigquery.Client(project='teamb09')

# set up some parameters
dataset_id = 'teamb09.web_scrape'
job_config = bigquery.LoadJobConfig(autodetect=True, write_disposition='WRITE_TRUNCATE')

In [53]:
for df, file in zip(dataframes, filenames):
    
    columns = df.columns
    data_types = ['STRING'] * len(columns)  # All columns are assumed to be 'STRING' type
    schema = [bigquery.SchemaField(name, data_type) for name, data_type in zip(columns, data_types)]

    table_id = f'{dataset_id}.{file}'
    table = bigquery.Table(table_id, schema=schema)
    client.create_table(table)
    client.load_table_from_dataframe(df, table_id, job_config=job_config)

    print(f'Loaded {table.num_rows} rows and {len(table.schema)} columns to {table_id}')

Loaded None rows and 12 columns to teamb09.web_scrape.rev_by_team
Loaded None rows and 12 columns to teamb09.web_scrape.ticket_revenue
Loaded None rows and 12 columns to teamb09.web_scrape.operating_income
Loaded None rows and 2 columns to teamb09.web_scrape.jersey_patch_sponsorships
